# Notebook to link trading bot to IB account

Latest version: 2024-08-25  
Author: MvS

## Description


## Result




### Testing, using official API client package

source: [IB](https://www.interactivebrokers.com/campus/ibkr-quant-news/interactive-brokers-python-api-native-a-step-by-step-guide/)

### Moving to Lumibot

In [1]:
from lumibot.brokers import InteractiveBrokers
from lumibot.backtesting import YahooDataBacktesting

from lumibot.traders import Trader
#from lumibot.strategies.examples import Strangle

from lumibot.strategies.strategy import Strategy
#from lumibot.entities import Asset

from datetime import datetime, timedelta
#from alpaca_trade_api import REST
from dotenv import dotenv_values

#from utils.finbert_utils import estimate_sentiment

env_dict = dotenv_values("../.env")

2024-08-26 14:18:32 | lumibot.credentials | WARNING | No .env file found. This is ok if you are using environment variables or secrets (like on Replit, AWS, etc), but if you are not, please create a .env file in the root directory of the project.


In [2]:
INTERACTIVE_BROKERS_CONFIG = {
    "IP": env_dict['IB_API_IP'],
    "SOCKET_PORT": int(env_dict['IB_API_PORT']),
    "CLIENT_ID": int(env_dict['IB_API_KEY']),
}

In [3]:
INTERACTIVE_BROKERS_CONFIG

{'IP': '192.168.178.81', 'SOCKET_PORT': 7497, 'CLIENT_ID': 357}

In [4]:
class IBTrader(Strategy):
    def initialize(self):
        print(f"INFO: initializing")
        self.sleeptime = "24H"
        self.last_trade = None

        cash = self.get_cash()
        print(f"INFO: position sizing, cash: {cash:10.3}")

        # self.api = REST(
        #     base_url=self.parameters["credentials"]["BASE_URL"],
        #     key_id=self.parameters["credentials"]["API_KEY"],
        #     secret_key=self.parameters["credentials"]["API_SECRET"],
        # )

    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.parameters["symbol"])
        quantity = round(cash * self.parameters["cash_at_risk"] / last_price, 0)

        return cash, last_price, quantity

    def get_dates(self):
        today = self.get_datetime()
        three_days_prior = today - timedelta(days=3)

        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')

    def get_sentiment(self):
        symbol = self.parameters["symbol"]
        print(f"INFO: getting news for {symbol}")

        today, three_days_prior = self.get_dates()
        #news = self.api.get_news(symbol=symbol, start=three_days_prior, end=today)

        #news = [ev.__dict__["_raw"]["headline"] for ev in news]
        #probability, sentiment = estimate_sentiment(news)

        #return probability, sentiment
        return 1.0, 'positive'

    def on_trading_iteration(self):
        print(f"INFO: trading loop")
        cash, last_price, quantity = self.position_sizing()
        probability, sentiment = self.get_sentiment()


In [9]:
broker = InteractiveBrokers(INTERACTIVE_BROKERS_CONFIG)
strat_dict = {
    "broker": broker,
    "force_start_immediately": True,
    "parameters": {
        "cash_at_risk": 0.03,
        "symbol": "EOAN",
        "trade_upside": 1.10,
        "trade_stop_loss": 0.95,
    },
}

strategy = IBTrader(**strat_dict)

# start_date = datetime(2022,6,1)
# end_date = start_date + timedelta(days=365*2)

# backtest_dict = {
#     "datasource_class": YahooDataBacktesting,
#     "backtesting_start": start_date,
#     "backtesting_end": end_date,
#     "benchmark_asset": Asset(symbol="AAPL", asset_type="stock")
# }

# strategy.backtest(
#     parameters=backtest_dict
# )

trader = Trader()
trader.add_strategy(strategy)
trader.run_all()

2024-08-26 14:29:54,389: root: INFO: The Interactive Brokers queue was empty or max time reached for account summary
2024-08-26 14:29:54,392: asyncio: ERROR: Unable to get balances (cash, portfolio value, etc) from broker. Please check your broker and your broker configuration.
2024-08-26 14:30:07,396: root: ERROR: The queue was empty or max time reached for positions
2024-08-26 14:30:07,398: root: ERROR: Error: IBWrapper returned an error with -1 error code 504 that says Not connected
2024-08-26 14:30:07,404: asyncio: INFO: [IBTrader] Executing the initialize lifecycle method
INFO: initializing
2024-08-26 14:30:07,452: root: WARNING: Had to reconnect to the broker. Sleeping for 5 seconds to make sure the connection is established.
2024-08-26 14:30:25,461: root: INFO: The Interactive Brokers queue was empty or max time reached for account summary
2024-08-26 14:30:25,465: asyncio: ERROR: Unable to get balances (cash, portfolio value, etc) from broker. Please check your broker and your b

Exception in thread IBTrader:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/mavost/GITHUB/urban-trading-analysis/python_env/lib/python3.10/site-packages/lumibot/strategies/strategy_executor.py", line 957, in run
    self._initialize()
  File "/home/mavost/GITHUB/urban-trading-analysis/python_env/lib/python3.10/site-packages/lumibot/strategies/strategy_executor.py", line 287, in func_output
    result = func_input(self, *args, **kwargs)
  File "/home/mavost/GITHUB/urban-trading-analysis/python_env/lib/python3.10/site-packages/lumibot/strategies/strategy_executor.py", line 355, in _initialize
    self.strategy.initialize(**safe_params_to_pass)
  File "/tmp/ipykernel_74558/3738722197.py", line 8, in initialize
TypeError: unsupported format string passed to NoneType.__format__


{'IBTrader': {}}

In [6]:
strat_dict

{'broker': <lumibot.brokers.interactive_brokers.InteractiveBrokers at 0x7f1d7591bbe0>,
 'name': 'somestrat',
 'force_start_immediately': True,
 'parameters': {'cash_at_risk': 0.03,
  'symbol': 'EOAN',
  'trade_upside': 1.1,
  'trade_stop_loss': 0.95}}